# Limit observation object creation

In [24]:
class Observable_Unique():
    _instances: dict[str, 'Observable_Unique'] = {}

    def __new__(cls, name):
        # print("__NEW__")
        if name in Observable_Unique._instances:
            # print("    EXISTS")
            return Observable_Unique._instances[name]
        else:
            # print("    NEW")
            return super(Observable_Unique, cls).__new__(cls)
    
    def __init__(self, name):
        # print("__INIT__")
        if hasattr(self, 'uid'):
            # print("    SKIPPING EXISTING")
            return
        # print("    INITIATING")
        self.uid = 'O' + name
        Observable_Unique._instances[name] = self

    def __repr__(self):
        return self.uid

In [7]:
Observable_Unique('a')

__NEW__
    NEW
__INIT__
    INITIATING


Oa

In [8]:
Observable_Unique._instances

{'a': Oa}

In [9]:
Observable_Unique('a')

__NEW__
    EXISTS
__INIT__
    SKIPPING EXISTING


Oa

In [10]:
Observable_Unique._instances

{'a': Oa}

In [11]:
Observable_Unique('b')
Observable_Unique('c')
Observable_Unique('b')

__NEW__
    NEW
__INIT__
    INITIATING
__NEW__
    NEW
__INIT__
    INITIATING
__NEW__
    EXISTS
__INIT__
    SKIPPING EXISTING


Ob

In [12]:
Observable_Unique._instances

{'a': Oa, 'b': Ob, 'c': Oc}

# Limit observation sequences

In [13]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import oom
from src.oom.DiscreteValuedOOM import DiscreteValuedOOM, get_matrices
from src.oom.observable import *

sys.modules['oom'] = oom

In [16]:
from src.oom.observable import ObsSequence

ObsSequence([Observable_Unique('a')])

__NEW__
    EXISTS
__INIT__
    SKIPPING EXISTING


TypeError: observations is not a sequence of strings or observables.

In [17]:
import sys
sys.getsizeof(['Oa', 'Ob', 'Oc', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa'])

152

In [18]:
sys.getsizeof([Observable(name[1:]) for name in ['Oa', 'Ob', 'Oc', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa']])

184

In [19]:
class Observable_NotUnique:
    def __init__(self, name):
        self.uid = 'O' + name

In [20]:
sys.getsizeof([Observable_NotUnique(name[1:]) for name in ['Oa', 'Ob', 'Oc', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa']])

184

In [29]:
mylist = [Observable_NotUnique(name[1:]) for name in ['Oa', 'Ob', 'Oc', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa']]
for item in mylist:
    print(id(item), sys.getsizeof(item))
print(sys.getsizeof(mylist))

2568660876752 48
2568660876992 48
2568659071536 48
2568661141504 48
2568661141936 48
2568661140592 48
2568661142512 48
2568661141312 48
2568661140976 48
2568661142656 48
2568661142368 48
2568661141744 48
184


In [30]:
mylist = [Observable_Unique(name[1:]) for name in ['Oa', 'Ob', 'Oc', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa']]
for item in mylist:
    print(id(item), sys.getsizeof(item))
print(sys.getsizeof(mylist))

2568660868832 48
2568660874448 48
2568660867248 48
2568660868832 48
2568660868832 48
2568660868832 48
2568660868832 48
2568660868832 48
2568660868832 48
2568660868832 48
2568660868832 48
2568660868832 48
184


In [40]:
from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)


##### Example call #####
mylist = ['Oa', 'Ob', 'Oc', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa']
print(total_size(mylist))
mylist = [Observable_NotUnique(name[1:]) for name in ['Oa', 'Ob', 'Oc', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa']]
print(total_size(mylist))
mylist = [Observable_Unique(name[1:]) for name in ['Oa', 'Ob', 'Oc', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa', 'Oa']]
print(total_size(mylist))

281
760
328
